In [201]:
#import statements
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from music21 import converter, instrument, note, chord, stream, duration
import glob
import pickle 
import os
from itertools import chain
import copy

In [202]:

def data_extractor(directory):
  """
    Summary:
    Function converts midi files to metadata and appends nested metadata lists into one large list
    composed of all the songs in the dataset.

    Parameters:
    directory: String representation of directroy where midi files are located

    Returns:
    list: sequential midi file metadata 
   """
  data = []
  for file in glob.glob(directory):
          mid = converter.parse(file)
          notes_to_parse = None
          

          try: 
              s2 = instrument.partitionByInstrument(mid)
              notes_to_parse = s2.parts[0].recurse() 
                 
          except: 
              notes_to_parse = mid.flat.notes

          for element in notes_to_parse:
              if isinstance(element, note.Note):
                  data.append([str(element.pitch),str(float(element.offset)), str(element.quarterLength)])
              elif isinstance(element, chord.Chord):
                  data.append(['.'.join(str(n) for n in element.normalOrder),str(float(element.offset)),str(element.quarterLength)])
  return data

data = data_extractor()
  

In [203]:
def make_dict(data, to_int = True):
  """
    Summary:
    Function maps either midi metadata into integers and integers back into midi metadata.

    Parameters:
    data: the data to be used as keys in a dictionary
    to_int: if the function maps from midi to int or int to midi
    
    Returns:
    dictionary: dictioanry with keys from data argument 
   """
  unnested_data = list(chain(*data))
  data_set = sorted(set(unnested_data))
  if to_int:
    notes_to_ints = dict((note,number) for number, note in enumerate(data_set))
  else:
    notes_to_ints = dict((number,note) for number, note in enumerate(data_set))

  return notes_to_ints


In [204]:
def str_to_int():
  """
  Summary:
  Function maps original midi list data into the correspondong integer value from the defined ictionary

  Returns:
  list: integers corresponding to midi metadata 
  """
  mapper = make_dict(data)
  mapped_sequence = copy.deepcopy(data)
  for i in range(len(mapped_sequence)):
    for j in range(len(mapped_sequence[i])):
      mapped_sequence[i][j] = mapper[mapped_sequence[i][j]]
      
  return mapped_sequence



In [205]:
def int_to_string(output):
  """
  Summary:
  Function maps original integers values back into the midi metadata

  Parameters:
  output: model output list that need converting back into midi metadata

  Returns:
  list: midi metadata
  """
  mapper = make_dict(data,to_int = False)
  mapped_sequence = output
  for i in range(len(mapped_sequence)):
    for j in range(len(mapped_sequence[i])):
      mapped_sequence[i][j] = mapper[mapped_sequence[i][j]]
      
  return mapped_sequence
  

In [206]:
def to_midi(metadata):
  """
  Summary:
  Takes midi metadata and converts it to a Music21 stream. The stream can then easily be converted into a midi file.

  Parameters:
  metadata: midi metadata (notes,chords,offsets,durations)

  Returns:
  list: Music21 stream object
  """
  s = stream.Stream()
  for notes,offsets,duration in metadata:
    if notes[0].isalpha():
      n = note.Note(notes)
      n.duration.quarterLength = float(duration)
      s.insert(offsets,n)
    else:
      chords = list(map(int,notes.split('.')))
      c = chord.Chord(chords)
      c.duration.quarterLength = float(duration)
      s.insert(offsets,c)
  return s
to_midi(int_to_string(str_to_int())).write("midi", "s.mid")


's.mid'